In [1]:
import pandas as pd
import numpy as np

from gensim.models import Word2Vec, KeyedVectors
from nltk.tokenize import RegexpTokenizer

import os
import pickle

seed = 42
np.random.seed(seed)

Preprocess Data (Splited by portion)

In [2]:
import re

#######
def text_cleansing(text):
    regex_except_token = r'\B(?!<\w+>\B)[^\w\s]'
    regex_expect_words = r'[^\w<>]+'
    output = re.sub(regex_except_token, '', text)
    output = re.sub(regex_expect_words, ' ', output)
    return output

struct_log = pd.read_csv("../Drain_result/thunderbird_10M.log_structured.csv")
template_log = pd.read_csv("../Drain_result/thunderbird_10M.log_templates.csv")

struct_log["Label"] = struct_log["Label"].apply(lambda x: int(x != "-"))
struct_log.sort_values("Time", inplace=True)
struct_log[struct_log["Label"] == 1].Date.value_counts().sort_index()
split_date = struct_log[struct_log.Label == 1].Date.values[0]

trainset = struct_log[struct_log.Date < split_date]
testset = struct_log[struct_log.Date >= split_date] 
eventid_train = trainset.EventId.unique() 
eventid_test = testset.EventId.unique() 

template_log_train = template_log[template_log["EventId"].isin(eventid_train)]
template_log_test = template_log[template_log["EventId"].isin(eventid_test)]
template_log_train["EventTemplateIdent_cleansed"] = template_log_train.EventTemplateIdent.map(text_cleansing)

template_log_train_list = template_log_train["EventTemplateIdent_cleansed"].astype('str').tolist()

/var/folders/jy/qhf5nsts0_77mx3gxd3731800000gn/T/ipykernel_83118/3606537976.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  template_log_train["EventTemplateIdent_cleansed"] = template_log_train.EventTemplateIdent.map(text_cleansing)


In [3]:
tokenizer = RegexpTokenizer(r'[A-Z][a-z]+|\w+')
token_train_list = [ tokenizer.tokenize(sen) for sen in template_log_train_list ]

template_log_train["EventTemplateIdent_token"] = pd.Series(token_train_list)
trainset["Token"] = trainset.EventId.map(
    lambda id: template_log_train[template_log_train.EventId == id].
    EventTemplateIdent_token.values[0]
    )

/var/folders/jy/qhf5nsts0_77mx3gxd3731800000gn/T/ipykernel_83118/1848025451.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  template_log_train["EventTemplateIdent_token"] = pd.Series(token_train_list)
/var/folders/jy/qhf5nsts0_77mx3gxd3731800000gn/T/ipykernel_83118/1848025451.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainset["Token"] = trainset.EventId.map(


In [ ]:
template_log_train.to_pickle("./output/BGL/processed/log_template.trainset.pkl")
trainset.to_pickle("./output/BGL/processed/log_structured.trainset.pkl")
testset.to_pickle("./output/BGL/processed/log_structured.testset.pkl")
template_log_test.to_pickle("./output/BGL/processed/log_structured.testset.pkl")

Preprocess Data (Splited by timeseries)

In [ ]:
from sklearn.model_selection import train_test_split

# clean special character
def text_cleansing(text):
    regex_except_token = r'\B(?!<\w+>\B)[^\w\s]'
    regex_expect_words = r'[^\w<>]+'
    output = re.sub(regex_except_token, '', text)
    output = re.sub(regex_expect_words, ' ', output)
    return output

struct_log = pd.read_csv("./output/BGL/BGL.log_structured.csv")
template_log = pd.read_csv("./output/BGL/BGL.log_templates.csv")

test_ratio = 0.4
struct_log["Label"] = struct_log["Label"].apply(lambda x: int(x != "-"))
struct_log.sort_values("Time", inplace=True)
struct_log[struct_log["Label"] == 1].Date.value_counts().sort_index()

trainset, testset = train_test_split(struct_log, test_size=0.4, random_state=seed, shuffle=False)
trainset = trainset[trainset["Label"] == 0]
eventid_train = trainset.EventId.unique()
eventid_test = testset.EventId.unique()

template_log_train = template_log[template_log["EventId"].isin(eventid_train)]
template_log_test = template_log[template_log["EventId"].isin(eventid_test)]
template_log_train["EventTemplateIdent_cleansed"] = template_log_train.EventTemplateIdent.map(text_cleansing)

template_log_train_list = template_log_train["EventTemplateIdent_cleansed"].astype('str').tolist()

In [ ]:
tokenizer = RegexpTokenizer(r'[A-Z][a-z]+|\w+')
token_train_list = [ tokenizer.tokenize(sen) for sen in template_log_train_list ]

template_log_train["EventTemplateIdent_token"] = pd.Series(token_train_list)
trainset["Token"] = trainset.EventId.map(
    lambda id: template_log_train[template_log_train.EventId == id].
    EventTemplateIdent_token.values[0]
    )

In [ ]:
template_log_train.to_pickle("./output/BGL/processed/log_template.trainset.pkl")
trainset.to_pickle("./output/BGL/processed/log_structured.trainset.pkl")
testset.to_pickle("./output/BGL/processed/log_structured.testset.pkl")
template_log_test.to_pickle("./output/BGL/processed/log_structured.testset.pkl")

In [4]:
# Download "GoogleNews-vectors-negative300.bin" from:
# https://www.kaggle.com/datasets/leadbest/googlenewsvectorsnegative300

model = KeyedVectors.load_word2vec_format(
    "../google_news/GoogleNews-vectors-negative300.bin",
    binary = True
    )

embedder = Word2Vec(vector_size=300, min_count=1)
embedder.build_vocab(token_train_list)
total_examples = embedder.corpus_count
embedder.build_vocab([list(model.key_to_index.keys())], update=True)

In [5]:
embedder.wv.vectors_lockf = np.ones(len(embedder.wv), dtype=np.float32)
embedder.wv.intersect_word2vec_format("../google_news/GoogleNews-vectors-negative300.bin", binary=True)

In [6]:
embedder.train(token_train_list, total_examples=total_examples, epochs=10)

(695519, 725400)

In [7]:
embedder.wv.save_word2vec_format("../thunderbird_10M-fine-tune-embedder.txt", binary=False)